In [3]:
import pandas as pd

In [20]:
team_data=pd.read_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\modelling_team_data_new.xlsx")

In [21]:
team_data.columns

Index(['Unnamed: 0', 'GAME_ID', 'SEASON', 'GAME_DATE_EST', 'OPP_TEAM_ID_x',
       'avg_pts', 'avg_fg_pct', 'avg_ft_pct', 'avg_fg3_pct', 'avg_ast',
       'avg_reb', 'avg_stl', 'avg_to', 'avg_pf', 'avg_fga', 'avg_efg_pct',
       'avg_fgm', 'win_streak', 'home_avg_pts', 'away_avg_pts', 'home_win_pct',
       'away_win_pct', 'home_head_to_head_avg_pts',
       'home_head_to_head_win_pct', 'OPP_TEAM_ID_y',
       'away_head_to_head_avg_pts', 'away_head_to_head_win_pct', 'opp_avg_pts',
       'opp_avg_fg_pct', 'opp_avg_ft_pct', 'opp_avg_fg3_pct', 'opp_avg_ast',
       'opp_avg_reb', 'opp_avg_stl', 'opp_avg_to', 'opp_avg_pf', 'opp_avg_fga',
       'opp_avg_efg_pct', 'opp_avg_fgm', 'opp_win_streak', 'opp_home_avg_pts',
       'opp_away_avg_pts', 'opp_home_win_pct', 'opp_away_win_pct', 'TEAM_WINS',
       'IS_HOME'],
      dtype='object')

In [22]:
import pandas as pd


# Function to calculate missed shots
def calculate_missed_shots(fga, fgm):
    return fga - fgm

# Calculate missed shots for both teams
team_data['missed_fg_opp_x'] = team_data.apply(lambda row: calculate_missed_shots(row['avg_fga'], row['avg_fgm']), axis=1)
team_data['missed_fg_opp_y'] = team_data.apply(lambda row: calculate_missed_shots(row['opp_avg_fga'], row['opp_avg_fgm']), axis=1)

# Function to calculate IBASIC
def calculate_ibasic(points, fg_pct, ft_pct, fg3_pct, ast, reb, missed_fg, to, pf):
    # Positive contributions to the team's basic efficiency index
    positive_contributions = points + fg_pct + ft_pct + fg3_pct + ast + reb
    
    # Negative contributions (missed field goals, turnovers, personal fouls)
    negative_contributions = missed_fg + to + pf

    # Subtract negative contributions from positive contributions
    ibasic = positive_contributions - negative_contributions
    
    return ibasic

# Applying the IBASIC calculation to both teams
team_data['IBASIC_opp_x'] = team_data.apply(lambda row: calculate_ibasic(
    row['avg_pts'], row['avg_fg_pct'], row['avg_ft_pct'], row['avg_fg3_pct'], row['avg_ast'], row['avg_reb'], row['missed_fg_opp_x'], row['avg_to'], row['avg_pf']), axis=1)

team_data['IBASIC_opp_y'] = team_data.apply(lambda row: calculate_ibasic(
    row['opp_avg_pts'], row['opp_avg_fg_pct'], row['opp_avg_ft_pct'], row['opp_avg_fg3_pct'], row['opp_avg_ast'], row['opp_avg_reb'], row['missed_fg_opp_y'], row['opp_avg_to'], row['opp_avg_pf']), axis=1)

# Function to calculate ICMPR
def calculate_icmpr(row):
    icmpr = 0
    
    # Positive comparisons (if team's stat is better, add 1)
    if row['IBASIC_opp_x'] > row['IBASIC_opp_y']:
        icmpr += 1
    if row['avg_pts'] > row['opp_avg_pts']:
        icmpr += 1
    if row['avg_fg_pct'] > row['opp_avg_fg_pct']:
        icmpr += 1
    if row['avg_ft_pct'] > row['opp_avg_ft_pct']:
        icmpr += 1
    if row['avg_fg3_pct'] > row['opp_avg_fg3_pct']:
        icmpr += 1
    if row['avg_ast'] > row['opp_avg_ast']:
        icmpr += 1
    if row['avg_reb'] > row['opp_avg_reb']:
        icmpr += 1
    
    # Negative comparisons (if opponent's stat is better, subtract 1)
    if row['IBASIC_opp_y'] > row['IBASIC_opp_x']:
        icmpr -= 1
    if row['opp_avg_pts'] > row['avg_pts']:
        icmpr -= 1
    if row['opp_avg_fg_pct'] > row['avg_fg_pct']:
        icmpr -= 1
    if row['opp_avg_ft_pct'] > row['avg_ft_pct']:
        icmpr -= 1
    if row['opp_avg_fg3_pct'] > row['avg_fg3_pct']:
        icmpr -= 1
    if row['opp_avg_ast'] > row['avg_ast']:
        icmpr -= 1
    if row['opp_avg_reb'] > row['avg_reb']:
        icmpr -= 1

    return icmpr

# Applying the ICMPR calculation
team_data['ICMPR'] = team_data.apply(calculate_icmpr, axis=1)

# Calculate IEXT for both teams
team_data['IEXT_opp_x'] = team_data['IBASIC_opp_x'] + team_data['ICMPR']
team_data['IEXT_opp_y'] = team_data['IBASIC_opp_y'] - team_data['ICMPR']

# Display the updated DataFrame with the new columns
print(team_data[['GAME_ID', 'IBASIC_opp_x', 'IBASIC_opp_y', 'ICMPR', 'IEXT_opp_x', 'IEXT_opp_y']])



        GAME_ID  IBASIC_opp_x  IBASIC_opp_y  ICMPR  IEXT_opp_x  IEXT_opp_y
0      10500003        75.905        98.006     -5      70.905     103.006
1      10500015        80.415        94.257     -3      77.415      97.257
2      10500042        75.361        94.843     -3      72.361      97.843
3      10500047        62.687        92.336     -5      57.687      97.336
4      10500074        67.836        92.045     -5      62.836      97.045
...         ...           ...           ...    ...         ...         ...
47745  22200414        93.797       109.461     -5      88.797     114.461
47746  22200428        97.774       113.880     -7      90.774     120.880
47747  22200450       115.128       118.611     -3     112.128     121.611
47748  22200460       107.232       123.196     -5     102.232     128.196
47749  22200476        92.060       118.444     -3      89.060     121.444

[47750 rows x 6 columns]


In [23]:
team_data

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,OPP_TEAM_ID_x,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,opp_away_win_pct,TEAM_WINS,IS_HOME,missed_fg_opp_x,missed_fg_opp_y,IBASIC_opp_x,IBASIC_opp_y,ICMPR,IEXT_opp_x,IEXT_opp_y
0,84,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,0.8,0,1,44.0,39.9,75.905,98.006,-5,70.905,103.006
1,85,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,0.8,1,1,44.6,40.6,80.415,94.257,-3,77.415,97.257
2,86,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0.8,0,0,45.4,41.3,75.361,94.843,-3,72.361,97.843
3,87,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0.8,1,0,46.4,40.2,62.687,92.336,-5,57.687,97.336
4,88,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0.8,0,0,42.8,39.6,67.836,92.045,-5,62.836,97.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,50469,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,0.6,0,1,47.5,45.9,93.797,109.461,-5,88.797,114.461
47746,50470,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,0.8,0,1,49.2,45.1,97.774,113.880,-7,90.774,120.880
47747,50471,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,0.8,0,0,39.4,43.7,115.128,118.611,-3,112.128,121.611
47748,50472,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,0.8,1,0,44.8,43.3,107.232,123.196,-5,102.232,128.196


In [24]:
min_value = team_data['IEXT_opp_x'].min()
max_value = team_data['IEXT_opp_x'].max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: 43.821
Maximum value: 143.02299999999997


In [25]:
min_value = team_data['ICMPR'].min()
max_value = team_data['ICMPR'].max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -7
Maximum value: 7


In [26]:

# Function to calculate True Shooting Percentage (TS%)
def calculate_ts_pct(points, fga, fta):
    ts_pct = points / (2 * (fga + 0.44 * fta)) if (fga + 0.44 * fta) != 0 else 0
    return ts_pct

# Calculate True Shooting Percentage for both teams
team_data['TS_pct_opp_x'] = team_data.apply(lambda row: calculate_ts_pct(row['avg_pts'], row['avg_fga'], row['avg_ft_pct'] * row['avg_fga']), axis=1)
team_data['TS_pct_opp_y'] = team_data.apply(lambda row: calculate_ts_pct(row['opp_avg_pts'], row['opp_avg_fga'], row['opp_avg_ft_pct'] * row['opp_avg_fga']), axis=1)

# Display the calculated TS% for both teams
print(team_data[['GAME_ID', 'TS_pct_opp_x', 'TS_pct_opp_y']])



        GAME_ID  TS_pct_opp_x  TS_pct_opp_y
0      10500003      0.461597      0.507039
1      10500015      0.447995      0.502016
2      10500042      0.438785      0.502007
3      10500047      0.410651      0.503210
4      10500074      0.466620      0.505708
...         ...           ...           ...
47745  22200414      0.471557      0.484036
47746  22200428      0.450300      0.498043
47747  22200450      0.526530      0.515485
47748  22200460      0.492420      0.526252
47749  22200476      0.443827      0.521469

[47750 rows x 3 columns]


In [27]:
team_data

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,OPP_TEAM_ID_x,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,IS_HOME,missed_fg_opp_x,missed_fg_opp_y,IBASIC_opp_x,IBASIC_opp_y,ICMPR,IEXT_opp_x,IEXT_opp_y,TS_pct_opp_x,TS_pct_opp_y
0,84,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,1,44.0,39.9,75.905,98.006,-5,70.905,103.006,0.461597,0.507039
1,85,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,1,44.6,40.6,80.415,94.257,-3,77.415,97.257,0.447995,0.502016
2,86,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0,45.4,41.3,75.361,94.843,-3,72.361,97.843,0.438785,0.502007
3,87,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0,46.4,40.2,62.687,92.336,-5,57.687,97.336,0.410651,0.503210
4,88,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0,42.8,39.6,67.836,92.045,-5,62.836,97.045,0.466620,0.505708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,50469,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,1,47.5,45.9,93.797,109.461,-5,88.797,114.461,0.471557,0.484036
47746,50470,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,1,49.2,45.1,97.774,113.880,-7,90.774,120.880,0.450300,0.498043
47747,50471,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,0,39.4,43.7,115.128,118.611,-3,112.128,121.611,0.526530,0.515485
47748,50472,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,0,44.8,43.3,107.232,123.196,-5,102.232,128.196,0.492420,0.526252


In [28]:
team_data = team_data.round(3)

In [29]:
team_data

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,OPP_TEAM_ID_x,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,IS_HOME,missed_fg_opp_x,missed_fg_opp_y,IBASIC_opp_x,IBASIC_opp_y,ICMPR,IEXT_opp_x,IEXT_opp_y,TS_pct_opp_x,TS_pct_opp_y
0,84,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,1,44.0,39.9,75.905,98.006,-5,70.905,103.006,0.462,0.507
1,85,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,1,44.6,40.6,80.415,94.257,-3,77.415,97.257,0.448,0.502
2,86,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0,45.4,41.3,75.361,94.843,-3,72.361,97.843,0.439,0.502
3,87,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0,46.4,40.2,62.687,92.336,-5,57.687,97.336,0.411,0.503
4,88,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0,42.8,39.6,67.836,92.045,-5,62.836,97.045,0.467,0.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,50469,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,1,47.5,45.9,93.797,109.461,-5,88.797,114.461,0.472,0.484
47746,50470,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,1,49.2,45.1,97.774,113.880,-7,90.774,120.880,0.450,0.498
47747,50471,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,0,39.4,43.7,115.128,118.611,-3,112.128,121.611,0.527,0.515
47748,50472,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,0,44.8,43.3,107.232,123.196,-5,102.232,128.196,0.492,0.526


In [31]:
team_data.drop(['missed_fg_opp_x','IEXT_opp_y','missed_fg_opp_y'],axis=1 , inplace=True)

In [32]:
team_data

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,OPP_TEAM_ID_x,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,opp_home_win_pct,opp_away_win_pct,TEAM_WINS,IS_HOME,IBASIC_opp_x,IBASIC_opp_y,ICMPR,IEXT_opp_x,TS_pct_opp_x,TS_pct_opp_y
0,84,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,0.8,0.8,0,1,75.905,98.006,-5,70.905,0.462,0.507
1,85,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,0.8,0.8,1,1,80.415,94.257,-3,77.415,0.448,0.502
2,86,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0.8,0.8,0,0,75.361,94.843,-3,72.361,0.439,0.502
3,87,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0.8,0.8,1,0,62.687,92.336,-5,57.687,0.411,0.503
4,88,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0.6,0.8,0,0,67.836,92.045,-5,62.836,0.467,0.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,50469,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,0.8,0.6,0,1,93.797,109.461,-5,88.797,0.472,0.484
47746,50470,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,0.8,0.8,0,1,97.774,113.880,-7,90.774,0.450,0.498
47747,50471,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,1.0,0.8,0,0,115.128,118.611,-3,112.128,0.527,0.515
47748,50472,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,1.0,0.8,1,0,107.232,123.196,-5,102.232,0.492,0.526


In [33]:
team_data.rename(columns={'IEXT_opp_x': 'IEXT_X_Y'}, inplace=True)

In [34]:
team_data

,Unnamed: 0,GAME_ID,SEASON,GAME_DATE_EST,OPP_TEAM_ID_x,avg_pts,avg_fg_pct,avg_ft_pct,avg_fg3_pct,avg_ast,...,opp_home_win_pct,opp_away_win_pct,TEAM_WINS,IS_HOME,IBASIC_opp_x,IBASIC_opp_y,ICMPR,IEXT_X_Y,TS_pct_opp_x,TS_pct_opp_y
0,84,10500003,2005,2005-10-11,1610612737,97.5,0.462,0.676,0.367,20.6,...,0.8,0.8,0,1,75.905,98.006,-5,70.905,0.462,0.507
1,85,10500015,2005,2005-10-12,1610612737,96.9,0.452,0.758,0.305,22.6,...,0.8,0.8,1,1,80.415,94.257,-3,77.415,0.448,0.502
2,86,10500042,2005,2005-10-17,1610612737,95.2,0.444,0.745,0.272,22.3,...,0.8,0.8,0,0,75.361,94.843,-3,72.361,0.439,0.502
3,87,10500047,2005,2005-10-18,1610612737,87.3,0.417,0.766,0.304,20.3,...,0.8,0.8,1,0,62.687,92.336,-5,57.687,0.411,0.503
4,88,10500074,2005,2005-10-22,1610612737,94.1,0.456,0.654,0.326,19.0,...,0.6,0.8,0,0,67.836,92.045,-5,62.836,0.467,0.506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47745,50469,22200414,2022,2022-12-14,1610612766,110.8,0.461,0.758,0.378,23.7,...,0.8,0.6,0,1,93.797,109.461,-5,88.797,0.472,0.484
47746,50470,22200428,2022,2022-12-16,1610612766,111.0,0.464,0.792,0.318,23.8,...,0.8,0.8,0,1,97.774,113.880,-7,90.774,0.450,0.498
47747,50471,22200450,2022,2022-12-18,1610612766,117.8,0.533,0.736,0.359,28.5,...,1.0,0.8,0,0,115.128,118.611,-3,112.128,0.527,0.515
47748,50472,22200460,2022,2022-12-19,1610612766,115.4,0.483,0.806,0.343,26.1,...,1.0,0.8,1,0,107.232,123.196,-5,102.232,0.492,0.526


In [35]:
team_data.to_excel(r"C:\Users\Daphne\OneDrive\Documents\NBA data set\modelling_NBA_team.xlsx")